In [ ]:
# Import the OpenCV library for image and video processing.
import cv2

# Import the os library to interact with the operating system, such as file paths and directories.
import os

# Import the MediaPipe library for hand tracking and gesture recognition.
import mediapipe as mp

# Import display, Image, and clear_output functions from IPython.display to handle image output in Jupyter Notebook.
from IPython.display import display, Image, clear_output


In [ ]:
# Define the path where collected images will be stored. 
# This directory contains raw images collected for the sign language detection project.
IMAGES_PATH = "images/collected"

# Define the path where labeled images will be saved. 
# This directory is intended for images that have been processed and labeled for training the model.
LABELED_IMAGES_PATH = "images/labeled"


In [ ]:
# Create a list of dictionaries to store label information for hand signs.
# Each dictionary contains a 'name' representing the sign gesture 
# and an 'id' that serves as a unique identifier for mapping purposes.
label_info = [
    {'name': 'hello', 'id': 1},       # Label for the 'hello' sign, assigned ID 1
    {'name': 'thanks', 'id': 2},      # Label for the 'thanks' sign, assigned ID 2
    {'name': 'yes', 'id': 3},         # Label for the 'yes' sign, assigned ID 3
    {'name': 'no', 'id': 4},          # Label for the 'no' sign, assigned ID 4
    {'name': 'iloveyou', 'id': 5}     # Label for the 'I love you' sign, assigned ID 5
]


In [ ]:
# Access the MediaPipe Hands module, which provides tools for hand tracking and gesture recognition.
mp_hands = mp.solutions.hands

# Initialize the Hands class with specified parameters for detection and tracking confidence.
# min_detection_confidence sets the threshold for detecting hands, while min_tracking_confidence
# controls the confidence level for tracking hands across frames.
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Access the drawing utilities from MediaPipe to facilitate the visualization of hand landmarks and connections.
mp_drawing = mp.solutions.drawing_utils


In [ ]:
# Check if the directory for labeled images already exists.
if not os.path.exists(LABELED_IMAGES_PATH):
    # If the directory does not exist, create it to store labeled images.
    os.makedirs(LABELED_IMAGES_PATH)


In [ ]:
def draw_bounding_box_with_label(frame, label_name):
    """
    Draws a bounding box around detected hands and labels them as one.
    
    Parameters:
    - frame: The image frame from the video stream where hand detection is performed.
    - label_name: The label to be displayed above the bounding box, indicating the recognized gesture.
    
    Returns:
    - The modified frame with the bounding box and label drawn on it, and a success flag.
    """
    
    # Convert the BGR frame to RGB format, as MediaPipe expects RGB input.
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the RGB frame to detect hands and retrieve landmarks.
    results = hands.process(rgb_frame)

    # Check if any hands are detected in the frame.
    if results.multi_hand_landmarks:
        # Initialize bounding box coordinates to extremes for later adjustments.
        x_min, y_min = frame.shape[1], frame.shape[0]  # Start with maximum values
        x_max, y_max = 0, 0  # Start with minimum values
        
        # Iterate through each detected hand's landmarks.
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                # Convert landmark coordinates from normalized values to pixel values.
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                
                # Update bounding box coordinates based on the detected landmarks.
                x_min = min(x_min, x)  # Find the leftmost point
                y_min = min(y_min, y)  # Find the topmost point
                x_max = max(x_max, x)  # Find the rightmost point
                y_max = max(y_max, y)  # Find the bottommost point

            # Draw the landmarks for the current hand on the frame.
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Draw a bounding box around all detected hands combined.
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Green box with thickness 2
        
        # Put the label above the bounding box.
        cv2.putText(frame, label_name, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Return the frame with bounding box and label, and a success flag (True).
        if x_min < x_max and y_min < y_max:
            return frame, True
        else:
            return frame, False  # Return False if bounding box is invalid.
    else:
        # Return False if no hands are detected.
        return frame, False

In [ ]:
def process_images_with_bounding_boxes():
    """
    Processes images for each label to add bounding boxes, labels, and save them.
    This function iterates through all the labels defined in label_info, 
    reads the corresponding images, adds bounding boxes and labels, 
    and saves the modified images in a specified directory.
    """
    
    # Loop through each label in the label_info list.
    for label in label_info:
        label_name = label['name']  # Get the name of the current label.
        label_path = os.path.join(IMAGES_PATH, label_name)  # Construct the path for the raw images of this label.
        labeled_label_path = os.path.join(LABELED_IMAGES_PATH, label_name)  # Construct the output path for labeled images.

        # Ensure the output directory exists for the labeled images of the current label.
        if not os.path.exists(labeled_label_path):
            os.makedirs(labeled_label_path)

        # Check if the directory for the raw images exists.
        if not os.path.exists(label_path):
            print(f"Directory {label_path} not found, skipping.")  # Log if the directory is missing.
            continue  # Skip to the next label if the directory is not found.

        # Loop through each image in the current label's directory.
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)  # Construct the full image path.
            frame = cv2.imread(image_path)  # Read the image using OpenCV.
            
            # Check if the image was read successfully.
            if frame is None:
                print(f"Could not read image {image_path}, skipping.")  # Log if the image could not be read.
                continue  # Skip to the next image if there was an error.

            # Draw bounding box and label on the frame using the previously defined function.
            frame_with_box, success = draw_bounding_box_with_label(frame, label_name)

            # Only save the image if the labeling was successful.
            if success:
                labeled_image_path = os.path.join(labeled_label_path, image_name)
                cv2.imwrite(labeled_image_path, frame_with_box)  # Save the labeled image to the specified path.
                print(f"Saved labeled image: {labeled_image_path}")  # Log the successful save operation.
            else:
                print(f"Skipping image {image_name} as labeling was unsuccessful.")  # Log if labeling was unsuccessful.

            # Display the image in a Jupyter Notebook environment (optional).
            try:
                # Encode the image as JPEG for display.
                _, jpeg = cv2.imencode('.jpg', frame_with_box)
                display(Image(data=jpeg.tobytes()))  # Display the image.
                clear_output(wait=True)  # Clear previous output to show the current image.
            except:
                pass  # Handle any exceptions, particularly in non-IPython environments.

In [ ]:
def main():
    """
    Main function to initialize and run the bounding box labeling process.
    """
    print("Starting image processing for bounding box labeling and saving...")
    process_images_with_bounding_boxes()
    print("Image processing and saving completed.")

In [ ]:
if __name__ == "__main__":
    main()